In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
import pickle

In [ ]:
pip install "tensorflow>=2.0.0"
pip install --upgrade tensorflow-hub

In [ ]:
columns = ["raised_amnt", "goal_amnt", "story"]
df = pd.read_csv('small_filtered_geotagged_surgery_campaigns.csv', usecols=columns, encoding='utf8')
print("Contents in csv file: ", df)
print(df.shape)

In [ ]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [ ]:
%time elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
def elmo_vectors( stories ):
    embeddings = elmo(stories.tolist(), signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # average along axis 1
        return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
batch_size = 4
list_train = [df[i:i+batch_size] for i in range(0,df.shape[0], batch_size)]

In [ ]:
elmo_train = [elmo_vectors(x["story"]) for x in list_train]

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis=0)

In [ ]:
print(elmo_train_new)
print(elmo_train_new.shape)

In [ ]:
pd.DataFrame(elmo_train_new).to_csv('encodings.csv') 